In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from dataclasses import dataclass
from math import radians
import time

import numpy as np
import pandas as pd
import numba as nb
import matplotlib.pyplot as plt
from tqdm.auto import trange
import ipywidgets as widgets
import xarray.ufuncs as xrf
from IPython.display import display, JSON

from cw.context import time_it
from cw.filters import smooth_signal

from topone.environments.launcher_v1 import LauncherV1, Stage, Simulation
from topone.environments.integrators import AB3Integrator
from topone.logger import Logger

# from environment import Environment
# from actor_critic_agent import ActorCriticAgent

In [3]:
logger = Logger()
env = LauncherV1(
    surface_diameter=1737.4e3,
    mu=4.9048695e12,
    stages=(
        Stage(
            dry_mass=1,
            propellant_mass=0.02,
            specific_impulse=100,
            thrust=2*1.7),
    ),
    initial_altitude=1000,
    initial_theta_e=radians(90),
    initial_longitude=radians(90),
)
logger.register_time_attribute(env.sim, "t")
logger.register(env.sim, "env1", ["h", "aii"])
logger.register(env.sim, "env2", ["engine_on", "xii"])

action = (True, False, 0.0)

In [8]:
with time_it("sim"):
    for j in trange(1000):
        env.reset()
        for i in range(1000-j*5):
            env.step(action)
            logger.step()
        logger.episode_finish()
        
batch_results = logger.batch_finish()

IndexError: deque index out of range

In [5]:
batch_results

<xarray.Dataset>
Dimensions:         (d_2_0: 2, episode_idx: 100)
Coordinates:
  * episode_idx     (episode_idx) int64 0 1 2 3 4 5 6 7 ... 93 94 95 96 97 98 99
Dimensions without coordinates: d_2_0
Data variables:
    env1_h          (episode_idx) float64 1.087e+03 1.087e+03 ... 1.022e+03
    env1_aii        (episode_idx, d_2_0) float64 1.119e-16 1.828 ... 1.769
    env2_engine_on  (episode_idx) bool True True True True ... True True True
    env2_xii        (episode_idx, d_2_0) float64 1.065e-10 ... 1.738e+06

In [8]:
plt.figure()
batch_results.env1_h.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
result = logger.log_to_dataset(logger.raw_episode_log, True)
result

<xarray.Dataset>
Dimensions:         (d_2_0: 2, t: 1000)
Coordinates:
  * t               (t) float64 0.01 0.02 0.03 0.04 0.05 ... 9.97 9.98 9.99 10.0
Dimensions without coordinates: d_2_0
Data variables:
    env1_h          (t) float64 1e+03 1e+03 1e+03 ... 1.087e+03 1.087e+03
    env1_aii        (t, d_2_0) float64 1.047e-16 1.71 ... 1.119e-16 1.828
    env2_engine_on  (t) bool True True True True True ... True True True True
    env2_xii        (t, d_2_0) float64 1.065e-10 1.738e+06 ... 1.738e+06

In [33]:
plt.figure()
result.env1_h.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plt.figure()
plt.plot(t_log[::100], h_log[::100], "-")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
JSON(env.sim_states_dict())

<IPython.core.display.JSON object>